## SAIL Turbulence Data from Town and Local Site

This notebook is used to investigate wind speed and temperature relationships over different time periods from the 2021-2022 winter data acquisition period. From documentation on the surface meteorological station from ARM, heights of measurement are Winds: 10 meters • Temperature and Relative Humidity: 2 meters • Barometric Pressure: 1 meter.

In [ ]:
# Import packages
import ftplib
from contextlib import closing
import urllib.request
import io
import os

import soslib.funcs as sos
import nctoolkit as nc
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import xarray as xr
import act

import holoviews as hv 
import hvplot.xarray
import panel as pn
import panel.widgets as pnw

hv.extension('bokeh')
from bokeh.plotting import show

Add host name, token and path to data

In [ ]:
username = os.getenv("ARM_USENAME")
token = os.getenv("ARM_TOKEN")
inpath = '/storage/dlhogan/sos/data'
outpath = '/storage/dlhogan/sos/data/ncdfs'

Choose location of data and start/end dates

In [ ]:
ecor_town = 'guc30ecorM1.b1'
ecor_kp = 'guc30ecorS3.b1'
surf_temp = 'gucgndirtM1.b1'
ceil = 'gucceilM1.b1'
sebs = 'gucsebsM1.b1'
start = '2022-01-05'
end = '2022-01-07' 

Download data to location if not already done

In [ ]:
# Download SAIL met data
if not os.path.exists(inpath+'/met'):
    print('Downloading Data...')
    act.discovery.download_data(username, token, met,
                                start, end, output=outpath+'/met')
    met_ds=act.io.armfiles.read_netcdf(os.path.join(outpath,f'met/*'))
    met_ds.to_netcdf(f'{outpath}/met_ds.nc')
else: 
    met_ds = act.io.armfiles.read_netcdf(os.path.join(outpath,'met_ds.nc'))
    print(f'Data already downloaded. Located here: {outpath}/met')


In [ ]:
# Download SAIL ecor data
if not os.path.exists(inpath+'/ecor_town'):
    print('Downloading Data...')
    act.discovery.download_data(username, token, ecor_town,
                                start, end, output=inpath+'/ecor_town')
    ecor_gts_ds=act.io.armfiles.read_netcdf(os.path.join(inpath,f'ecor_gts/*'))
    ecor_gts_ds.to_netcdf(f'{outpath}/ecor_gts.nc')
else: 
    ecor_gts_ds = act.io.armfiles.read_netcdf(os.path.join(outpath,'ecor_gts_ds.nc'))
    print(f'Data already downloaded. Located here: {inpath}/ecor_town')


In [ ]:
# Download SAIL ecor data
if not os.path.exists(inpath+'/ecor_kp'):
    print('Downloading Data...')
    act.discovery.download_data(username, token, ecor_kp,
                                start, end, output=inpath+'/ecor_kp')
else: 
    print(f'Data already downloaded. Located here: {inpath}/ecor_kp')

In [ ]:
# Download SAIL ecor data
if not os.path.exists(inpath+'/surf_temp'):
    print('Downloading Data...')
    act.discovery.download_data(username, token, surf_temp,
                                start, end, output=inpath+'/surf_temp')
    surf_temp_ds=act.io.armfiles.read_netcdf(os.path.join(inpath,f'surf_temp/*'))
    surf_temp_ds.to_netcdf(f'{outpath}/surf_temp.nc')
else: 
    surf_temp_ds=act.io.armfiles.read_netcdf(os.path.join(outpath,f'surf_temp_ds.nc'))
    print(f'Data already downloaded. Located here: {inpath}/surf_temp')

In [ ]:
# Download SAIL ecor data
if not os.path.exists(inpath+'/sebs'):
    print('Downloading Data...')
    act.discovery.download_data(username, token, sebs,
                                start, end, output=inpath+'/sebs')
    sebs_gts_ds=act.io.armfiles.read_netcdf(os.path.join(inpath,f'sebs/*'))
    sebs_gts_ds.to_netcdf(f'{outpath}/sebs.nc')
else: 
    sebs_gts_ds=act.io.armfiles.read_netcdf(os.path.join(outpath,f'sebs_gts_ds.nc'))
    print(f'Data already downloaded. Located here: {inpath}/sebs')

Create dataset for investigation in remainder of notebook:

In [ ]:
# Surface Temperature
surf_t = surf_temp_ds.where(surf_temp_ds['qc_sfc_ir_temp']==0)['sfc_ir_temp']

In [ ]:
# Downwelling shortwave radiation
sw_rad = sebs_gts_ds.where(sebs_gts_ds['qc_down_short_hemisp']==0)[['down_short_hemisp','up_short_hemisp']]

In [ ]:
# Choose a time slice
time_slice = slice('2022-04-21','2022-04-25')
met_ds_subset = met_ds.sel(time=time_slice)

## Plot turbulence regimes from Sun et al 2012

In [ ]:

time_slice = slice('2022-01-01','2022-01-04')
ecor_ds_subset = ecor_gts_ds.sel(time=time_slice)
v_tke = (0.5 * (ecor_ds_subset['var_u']**2 + ecor_ds_subset['var_v']**2 + ecor_ds_subset['var_w']**2))**0.5
wind_speed = ecor_ds_subset.where(ecor_ds_subset['qc_wind_spd']==0)['wind_spd']

fig, ax = plt.subplots(figsize=(8,6))
fig.set_facecolor('white')

ax.scatter(wind_speed, v_tke)

ax.axvline(2,label='Saltating Snow\nTheshold', color='magenta', lw=4, ls='-.')
ax.axvline(5,label='Blowing Snow\nThreshold', color='orange', lw=4, ls='--')

ax.set_ylim(0,6)
ax.set_xlim(0,6)
ax.tick_params(axis='both', labelsize=14)
ax.set_xlabel('Wind Speed (m/s)',size=14)
ax.set_ylabel('Turbulence Intensity (m/s)', size=14)
ax.set_title('Turbulence Regimes @ 3m: Jan 1-4 2022', size=18)
ax.legend(loc='upper right',fontsize=14)
time_slice = slice('2022-02-10','2022-02-14')
ecor_ds_subset = ecor_gts_ds.sel(time=time_slice)
v_tke = (0.5 * (ecor_ds_subset['var_u']**2 + ecor_ds_subset['var_v']**2 + ecor_ds_subset['var_w']**2))**0.5
wind_speed = ecor_ds_subset.where(ecor_ds_subset['qc_wind_spd']==0)['wind_spd']

fig, ax = plt.subplots(figsize=(8,6))
fig.set_facecolor('white')

ax.scatter(wind_speed, v_tke)

ax.axvline(2,label='Saltating Snow\nTheshold', color='magenta', lw=4, ls='-.')
ax.axvline(5,label='Blowing Snow\nThreshold', color='orange', lw=4, ls='--')

ax.set_ylim(0,6)
ax.set_xlim(0,6)
ax.tick_params(axis='both', labelsize=14)
ax.set_xlabel('Wind Speed (m/s)',size=14)
ax.set_ylabel('Turbulence Intensity (m/s)', size=14)
ax.set_title('Turbulence Regimes @ 3m: Feb 10-14 2022', size=18)
ax.legend(loc='upper right',fontsize=14)

In [ ]:
time_slice = slice('2021-12-22','2021-12-31')
ecor_ds_subset = ecor_gts_ds.sel(time=time_slice)
v_tke = (0.5 * (ecor_ds_subset['var_u']**2 + ecor_ds_subset['var_v']**2 + ecor_ds_subset['var_w']**2))**0.5
wind_speed_ = met_ds.sel(time=time_slice).resample(time='30min').mean()['wspd_arith_mean']
wind_speed = wind_speed_.sel(time=v_tke.time)

fig, axs = plt.subplots(ncols=3, figsize=(16,5), sharey=True, tight_layout=True)

fig.set_facecolor('white')

# Plot 1
axs[0].scatter(wind_speed, v_tke)

axs[0].axvline(2,label='Saltating Snow\nTheshold', color='magenta', lw=4, ls='-.')
axs[0].axvline(5,label='Blowing Snow\nThreshold', color='orange', lw=4, ls='--')

axs[0].set_ylim(0,6)
axs[0].set_xlim(0,8)
axs[0].tick_params(axis='both', labelsize=14)
axs[0].set_xlabel('30-min Average Wind Speed (m/s)',size=14)
axs[0].set_ylabel('Turbulence Intensity (m/s)', size=14)
axs[0].set_title('"Santa Slammer"\nDec 22-31 2021', size=18)
# axs[0].legend(loc='upper right',fontsize=14)

# Plot 2
time_slice = slice('2022-01-20','2022-01-26')
ecor_ds_subset = ecor_gts_ds.sel(time=time_slice)
v_tke = (0.5 * (ecor_ds_subset['var_u']**2 + ecor_ds_subset['var_v']**2 + ecor_ds_subset['var_w']**2))**0.5
wind_speed = ecor_ds_subset.where(ecor_ds_subset['qc_wind_spd']==0)['wind_spd']

axs[1].scatter(wind_speed, v_tke)

axs[1].axvline(2,label='Saltating Snow\nTheshold', color='magenta', lw=4, ls='-.')
axs[1].axvline(5,label='Blowing Snow\nThreshold', color='orange', lw=4, ls='--')

axs[1].set_ylim(0,6)
axs[1].set_xlim(0,8)
axs[1].tick_params(axis='both', labelsize=14)
axs[1].set_xlabel('30-min Average Wind Speed (m/s)',size=14)
# axs[1].set_ylabel('Turbulence Intensity (m/s)', size=14)
axs[1].set_title('Quiescent Period\nJan 20-26 2022', size=18)
axs[1].legend(ncol=2, loc='upper center',fontsize=14, framealpha=1)

# Plot 3
time_slice = slice('2022-02-10','2022-02-14')
ecor_ds_subset = ecor_gts_ds.sel(time=time_slice)
v_tke = (0.5 * (ecor_ds_subset['var_u']**2 + ecor_ds_subset['var_v']**2 + ecor_ds_subset['var_w']**2))**0.5
wind_speed_ = met_ds.sel(time=time_slice).resample(time='30min').mean()['wspd_arith_mean']
wind_speed = wind_speed_.sel(time=v_tke.time)

axs[2].scatter(wind_speed, v_tke)

axs[2].axvline(2,label='Saltating Snow\nTheshold', color='magenta', lw=4, ls='-.')
axs[2].axvline(5,label='Blowing Snow\nThreshold', color='orange', lw=4, ls='--')

axs[2].set_ylim(0,6)
axs[2].set_xlim(0,8)
axs[2].tick_params(axis='both', labelsize=14)
axs[2].set_xlabel('30-min Average Wind Speed (m/s)',size=14)
# axs[2].set_ylabel('Turbulence Intensity (m/s)', size=14)
axs[2].set_title('Windy Period\nFeb 10-14 2022', size=18)
# axs[2].legend(loc='upper right',fontsize=14)

fig.suptitle('Turbulence Regimes from SAIL ECOR (30-minute Ensemble Means)', fontsize=18)


In [ ]:
time_slice = slice('2022-03-09','2022-03-14')
ecor_ds_subset = ecor_gts_ds.sel(time=time_slice)
v_tke = (0.5 * (ecor_ds_subset['var_u']**2 + ecor_ds_subset['var_v']**2 + ecor_ds_subset['var_w']**2))**0.5
wind_speed = ecor_ds_subset.where(ecor_ds_subset['qc_wind_spd']==0)['wind_spd']

fig, ax = plt.subplots(figsize=(8,6))
ax.scatter(wind_speed, v_tke)

ax.axvline(2,label='Saltating Snow\nTheshold', color='magenta', lw=4, ls='-.')
ax.axvline(5,label='Blowing Snow\nThreshold', color='orange', lw=4, ls='--')

ax.set_ylim(0,6)
ax.set_xlim(0,6)
ax.tick_params(axis='both', labelsize=14)
ax.set_xlabel('Wind Speed (m/s)',size=14)
ax.set_ylabel('Turbulence Intensity (m/s)', size=14)
ax.set_title('Turbulence Regimes @ 3m: Mar 9-14 2022', size=18)
ax.legend(loc='upper right',fontsize=14)

In [ ]:
from matplotlib.colors import Normalize 
from scipy.interpolate import interpn
from matplotlib import cm


def density_scatter( x , y, ax = None, sort = True, bins = 20, **kwargs )   :
    """
    Scatter plot colored by 2d histogram
    """
    if ax is None :
        fig , ax = plt.subplots(figsize=(8,6))
    data , x_e, y_e = np.histogram2d( x, y, bins = bins, density = True )
    z = interpn( ( 0.5*(x_e[1:] + x_e[:-1]) , 0.5*(y_e[1:]+y_e[:-1]) ) , data , np.vstack([x,y]).T , method = "splinef2d", bounds_error = False)

    #To be sure to plot all data
    z[np.where(np.isnan(z))] = 0.0

    # Sort the points by density, so that the densest points are plotted last
    if sort :
        idx = z.argsort()
        x, y, z = x[idx], y[idx], z[idx]

    ax.scatter( x, y, c=z, **kwargs )

    norm = Normalize(vmin = 0, vmax = np.max(z))
    cbar = fig.colorbar(cm.ScalarMappable(cmap='viridis', norm = norm),ax=ax)
    cbar.ax.set_ylabel('Density', size=18)
    ax.axvline(2,label='Saltating Snow\nTheshold', color='magenta', lw=4, ls='-.')
    ax.axvline(5,label='Blowing Snow\nThreshold', color='orange', lw=4, ls='--')

    ax.set_ylim(0,6)
    ax.set_xlim(0,6)
    ax.tick_params(axis='both', labelsize=14)
    ax.set_xlabel('Wind Speed (m/s)',size=14)
    ax.set_ylabel('Turbulence Intensity (m/s)', size=14)
    ax.set_title(f'Turbulence Regimes @ 3m: {t1} to {t2}', size=18)
    ax.legend(loc='upper right',fontsize=14)

    return ax

In [ ]:
t1 = '2022-01-01'
t2 = '2022-01-04'

def turbulent_regime_plot(t1,t2):
    ecor_ds_subset = ecor_gts_ds.sel(time=slice(t1,t2))
    v_tke = (0.5 * (ecor_ds_subset['var_u']**2 + ecor_ds_subset['var_v']**2 + ecor_ds_subset['var_w']**2))**0.5
    wind_speed = ecor_ds_subset.where(ecor_ds_subset['qc_wind_spd']==0)['wind_spd']


    x = np.nan_to_num(wind_speed.values)
    y = np.nan_to_num(v_tke.values)

    density_scatter( x, y)
    return

In [ ]:
t1 = '2021-12-22'
t2 = '2021-12-31'
turbulent_regime_plot(t1,t2)

t1 = '2022-02-10'
t2 = '2022-02-14'
turbulent_regime_plot(t1,t2)

t1 = '2022-03-09'
t2 = '2022-03-14'
turbulent_regime_plot(t1,t2)

## Plot turbulent fluxes with time

In [ ]:
ecor_gts_ds

In [ ]:
default_settings = {'height': 200, 'width': 500}

time_slice = slice('2022-04-21','2022-04-25')
sebs_gts_ds_subset = sw_rad.sel(time=time_slice)
surf_temp_da_subset = surf_t.sel(time=time_slice)
ecor_ds_subset = ecor_gts_ds.sel(time=time_slice)
met_ds_subset = met_ds.sel(time=time_slice)

sw_down = (sebs_gts_ds_subset['down_short_hemisp']/10).hvplot(label='Downwelling SW/10',**default_settings)
lv = ecor_ds_subset.where(ecor_ds_subset['qc_lv_e']==0)['lv_e'].hvplot(label='Latent Heat Flux',
                                                                      **default_settings)
hs = ecor_ds_subset.where(ecor_ds_subset['qc_h']==0)['h'].hvplot(label='Sensible Heat Flux',
                                                                 ylabel='Sensible/Latent Heat Flux\n(W/m^2)',
                                                                 **default_settings)
temp = (met_ds_subset.where(met_ds_subset['qc_temp_mean']==0)['temp_mean']).hvplot(label='Temperature (2m)',
                                                                                ylabel='Temperature (C)',
                                                                                **default_settings,
                                                                                color='orange')          
surf_temp_plt = (surf_temp_da_subset-273.15).hvplot(label='Surface Temperature')                                                                                                                                                                                                                       
wind_spd = met_ds_subset.where(met_ds_subset['qc_wspd_arith_mean']==0)['wspd_arith_mean'].hvplot(label='Wind Speed',
                                                                                  ylabel='Wind Speed (m/s)',
                                                                                  **default_settings,
                                                                                  color='green')       
southern_winds=(ecor_ds_subset.where((ecor_ds_subset['wind_dir']>90) & (ecor_ds_subset['wind_dir']<270))['wind_dir']).hvplot.scatter(label='Wind Direction (SW-SE winds)',
                                                                                  ylabel='Degrees',
                                                                                  **default_settings,
                                                                                  color='black').opts(marker='x',legend_position='top', legend_offset=(0,5),legend_cols=1) 
northern_winds1=(ecor_ds_subset.where((ecor_ds_subset['wind_dir']>270))['wind_dir']-360).hvplot.scatter(
                                                                                  ylabel='Degrees',
                                                                                  **default_settings,
                                                                                  color='red').opts(marker='x',legend_position='top', legend_offset=(0,5),legend_cols=1)
northern_winds2=(ecor_ds_subset.where((ecor_ds_subset['wind_dir']<90))['wind_dir']).hvplot.scatter(label='Wind Direction (NW-NE winds)',
                                                                                  ylabel='Degrees',
                                                                                  **default_settings,
                                                                                  color='red').opts(marker='x',legend_position='top', legend_offset=(0,5),legend_cols=1)                                                                                                
((lv*hs*sw_down).opts(legend_position='top', legend_offset=(0,5),legend_cols=1)+(temp*surf_temp_plt).opts(legend_position='top', legend_offset=(0,5),legend_cols=1)+wind_spd+northern_winds1*southern_winds*northern_winds2).cols(2)                         

In [ ]:
default_settings = {'height': 200, 'width': 500}

time_slice = slice('2022-03-08','2022-03-13')
sebs_gts_ds_subset = sw_rad.sel(time=time_slice)
surf_temp_da_subset = surf_t.sel(time=time_slice)
ecor_ds_subset = ecor_gts_ds.sel(time=time_slice)
met_ds_subset = met_ds.sel(time=time_slice)

sw_down = (sebs_gts_ds_subset['down_short_hemisp']/10).hvplot(label='Downwelling SW/10',**default_settings)
lv = ecor_ds_subset.where(ecor_ds_subset['qc_lv_e']==0)['lv_e'].hvplot(label='Latent Heat Flux',
                                                                      **default_settings)
hs = ecor_ds_subset.where(ecor_ds_subset['qc_h']==0)['h'].hvplot(label='Sensible Heat Flux',
                                                                 ylabel='Sensible/Latent Heat Flux\n(W/m^2)',
                                                                 **default_settings)
temp = (met_ds_subset.where(met_ds_subset['qc_temp_mean']==0)['temp_mean']).hvplot(label='Temperature',
                                                                                ylabel='Temperature (C)',
                                                                                **default_settings,
                                                                                color='orange')          
surf_temp_plt = (surf_temp_da_subset-273.15).hvplot(label='Surface Temperature')                                                                                                                                                                                                                       
wind_spd = met_ds_subset.where(met_ds_subset['qc_wspd_arith_mean']==0)['wspd_arith_mean'].hvplot(label='Wind Speed',
                                                                                  ylabel='Wind Speed (m/s)',
                                                                                  **default_settings,
                                                                                  color='green')       
southern_winds=(ecor_ds_subset.where((ecor_ds_subset['wind_dir']>90) & (ecor_ds_subset['wind_dir']<270))['wind_dir']).hvplot.scatter(label='Wind Direction (SW-SE winds)',
                                                                                  ylabel='Degrees',
                                                                                  **default_settings,
                                                                                  color='black').opts(marker='x',legend_position='top', legend_offset=(0,5),legend_cols=1) 
northern_winds1=(ecor_ds_subset.where((ecor_ds_subset['wind_dir']>270))['wind_dir']-360).hvplot.scatter(
                                                                                  ylabel='Degrees',
                                                                                  **default_settings,
                                                                                  color='red').opts(marker='x',legend_position='top', legend_offset=(0,5),legend_cols=1)
northern_winds2=(ecor_ds_subset.where((ecor_ds_subset['wind_dir']<90))['wind_dir']).hvplot.scatter(label='Wind Direction (NW-NE winds)',
                                                                                  ylabel='Degrees',
                                                                                  **default_settings,
                                                                                  color='red').opts(marker='x',legend_position='top', legend_offset=(0,5),legend_cols=1)                                                                                                
((lv*hs*sw_down).opts(legend_position='top', legend_offset=(0,5),legend_cols=1)+(temp*surf_temp_plt).opts(legend_position='top', legend_offset=(0,5),legend_cols=1)+wind_spd+northern_winds1*southern_winds*northern_winds2).cols(2)                         

In [ ]:
default_settings = {'height': 200, 'width': 500}

time_slice = slice('2021-12-09','2021-12-15')
sebs_gts_ds_subset = sw_rad.sel(time=time_slice)
surf_temp_da_subset = surf_t.sel(time=time_slice)
ecor_ds_subset = ecor_gts_ds.sel(time=time_slice)
met_ds_subset = met_ds.sel(time=time_slice)

sw_down = (sebs_gts_ds_subset['down_short_hemisp']/10).hvplot(label='Downwelling SW/10',**default_settings)
lv = ecor_ds_subset.where(ecor_ds_subset['qc_lv_e']==0)['lv_e'].hvplot(label='Latent Heat Flux',
                                                                      **default_settings)
hs = ecor_ds_subset.where(ecor_ds_subset['qc_h']==0)['h'].hvplot(label='Sensible Heat Flux',
                                                                 ylabel='Sensible/Latent Heat Flux\n(W/m^2)',
                                                                 **default_settings)
temp = (met_ds_subset.where(met_ds_subset['qc_temp_mean']==0)['temp_mean']).hvplot(label='Temperature',
                                                                                ylabel='Temperature (C)',
                                                                                **default_settings,
                                                                                color='orange')          
surf_temp_plt = (surf_temp_da_subset-273.15).hvplot(label='Surface Temperature')                                                                                                                                                                                                                       
wind_spd = met_ds_subset.where(met_ds_subset['qc_wspd_arith_mean']==0)['wspd_arith_mean'].hvplot(label='Wind Speed',
                                                                                  ylabel='Wind Speed (m/s)',
                                                                                  **default_settings,
                                                                                  color='green')       
southern_winds=(ecor_ds_subset.where((ecor_ds_subset['wind_dir']>90) & (ecor_ds_subset['wind_dir']<270))['wind_dir']).hvplot.scatter(label='Wind Direction (SW-SE winds)',
                                                                                  ylabel='Degrees',
                                                                                  **default_settings,
                                                                                  color='black').opts(marker='x',legend_position='top', legend_offset=(0,5),legend_cols=1) 
northern_winds1=(ecor_ds_subset.where((ecor_ds_subset['wind_dir']>270))['wind_dir']-360).hvplot.scatter(
                                                                                  ylabel='Degrees',
                                                                                  **default_settings,
                                                                                  color='red').opts(marker='x',legend_position='top', legend_offset=(0,5),legend_cols=1)
northern_winds2=(ecor_ds_subset.where((ecor_ds_subset['wind_dir']<90))['wind_dir']).hvplot.scatter(label='Wind Direction (NW-NE winds)',
                                                                                  ylabel='Degrees',
                                                                                  **default_settings,
                                                                                  color='red').opts(marker='x',legend_position='top', legend_offset=(0,5),legend_cols=1)                                                                                                
((lv*hs*sw_down).opts(legend_position='top', legend_offset=(0,5),legend_cols=1)+(temp*surf_temp_plt).opts(legend_position='top', legend_offset=(0,5),legend_cols=1)+wind_spd+northern_winds1*southern_winds*northern_winds2).cols(2)                         

In [ ]:
ftp = ftplib.FTP('ftp1.esrl.noaa.gov') # Connect to NOAA's FTP
ftp.login() # Anonymous login
ftp.cwd('./Observations/Campaigns/SPLASH/asfs30/1_level_ingest')

files = ftp.nlst() # Collect files into vector
ftp.close()

turbsled_files = []
for file in files:
    if ('10min') in file and (('20211206') in file or ('20211207') in file or ('20211208') in file or ('20211209') in file):
        turbsled_files.append(file)
    

for i, file in enumerate(turbsled_files):
    print(f'Loading {file}...')
    # used this as reference: https://towardsdatascience.com/an-efficient-way-to-read-data-from-the-web-directly-into-python-a526a0b4f4cb
    url = 'ftp://ftp1.esrl.noaa.gov/Observations/Campaigns/SPLASH/asfs30/1_level_ingest/' + file

    req = urllib.request.Request(url)

    with urllib.request.urlopen(req) as resp:
        if i == 0:
            ds = xr.open_dataset(io.BytesIO(resp.read()),engine='h5netcdf')
        if i != 0:
            with urllib.request.urlopen(req) as resp:
                tmp = xr.open_dataset(io.BytesIO(resp.read()),engine='h5netcdf')
            ds = xr.merge([ds,tmp], compat='override')

In [ ]:
default_settings = {'height': 200, 'width': 500}

time_slice = slice('2021-12-06','2021-12-09')
sebs_gts_ds_subset = sw_rad.sel(time=time_slice)
surf_temp_da_subset = surf_t.sel(time=time_slice)
ecor_ds_subset = ecor_gts_ds.sel(time=time_slice)
met_ds_subset = met_ds.sel(time=time_slice)

lv_asfs = ds['bulk_Hl'].hvplot(label='Hl_ASFS')
hs_asfs = ds['bulk_Hs'].hvplot(label='Hs_ASFS')


sw_down = (sebs_gts_ds_subset['down_short_hemisp']/10).hvplot(label='Downwelling SW/10',**default_settings)
lv = ecor_ds_subset.where(ecor_ds_subset['qc_lv_e']==0)['lv_e'].hvplot(label='Hl',
                                                                      **default_settings)
hs = ecor_ds_subset.where(ecor_ds_subset['qc_h']==0)['h'].hvplot(label='Hs',
                                                                 ylabel='Sensible/Latent Heat Flux\n(W/m^2)',
                                                                 **default_settings)
temp = (met_ds_subset.where(met_ds_subset['qc_temp_mean']==0)['temp_mean']).hvplot(label='Temperature (2m)',
                                                                                ylabel='Temperature (C)',
                                                                                **default_settings,
                                                                                color='orange')          
surf_temp_plt = (surf_temp_da_subset-273.15).hvplot(label='Surface Temperature')                                                                                                                                                                                                                       
wind_spd = met_ds_subset.where(met_ds_subset['qc_wspd_arith_mean']==0)['wspd_arith_mean'].hvplot(label='Wind Speed',
                                                                                  ylabel='Wind Speed (m/s)',
                                                                                  **default_settings,
                                                                                  color='green')       
southern_winds=(ecor_ds_subset.where((ecor_ds_subset['wind_dir']>90) & (ecor_ds_subset['wind_dir']<270))['wind_dir']).hvplot.scatter(label='Wind Direction (SW-SE winds)',
                                                                                  ylabel='Degrees',
                                                                                  **default_settings,
                                                                                  color='black').opts(marker='x',legend_position='top', legend_offset=(0,5),legend_cols=1) 
northern_winds1=(ecor_ds_subset.where((ecor_ds_subset['wind_dir']>270))['wind_dir']-360).hvplot.scatter(
                                                                                  ylabel='Degrees',
                                                                                  **default_settings,
                                                                                  color='red').opts(marker='x',legend_position='top', legend_offset=(0,5),legend_cols=1)
northern_winds2=(ecor_ds_subset.where((ecor_ds_subset['wind_dir']<90))['wind_dir']).hvplot.scatter(label='Wind Direction (NW-NE winds)',
                                                                                  ylabel='Degrees',
                                                                                  **default_settings,
                                                                                  color='red').opts(marker='x',legend_position='top', legend_offset=(0,5),legend_cols=1)                                                                                                
((lv*hs*lv_asfs*hs_asfs).opts(legend_position='top', legend_offset=(0,5),legend_cols=1)+(temp*surf_temp_plt).opts(legend_position='top', legend_offset=(0,5),legend_cols=1)+wind_spd+northern_winds1*southern_winds*northern_winds2).cols(2)                         

## Playing with water vapor and temperature differences


In [ ]:
def sat_vapor_pressure(T):
    es_surf = 611.21 * np.exp((22.587 * T) / (T+273.86)) # Magnus formula from Huang 2018
    return es_surf

def sat_vapor_pressure_water(T):
    es_surf = 610.94 * np.exp((17.625*T) / (T + 243.04)) # Magnus formula from Huang 2018
    return es_surf

In [ ]:
temp_diff = met_ds['temp_mean'] - (surf_temp_ds['sfc_ir_temp'].where(surf_temp_ds['sfc_ir_temp']<=273.15)-273.15)
rh_mean = met_ds['rh_mean'].sel(time=temp_diff.time)/100

t_surf = surf_temp_ds['sfc_ir_temp'].where(surf_temp_ds['sfc_ir_temp']<=273.15)-273.15
meas_temp = met_ds['temp_mean'].compute()
rh = met_ds['rh_mean']
e_air = rh * sat_vapor_pressure_water(meas_temp)/100
es_surf = sat_vapor_pressure(t_surf)

In [ ]:
vpd = (e_air - es_surf)
wspd_arith_mean = met_ds['wspd_arith_mean']
u_vpd = wspd_arith_mean * vpd

In [ ]:
import matplotlib
import matplotlib.dates as mdates
from matplotlib.colors import Normalize

In [ ]:
fig, axs = plt.subplots(4, figsize=(8,16), tight_layout=True)
time_slice = [slice('2021-12-01','2021-12-31'),
              slice('2022-01-01','2022-01-31'),
              slice('2022-02-01','2022-02-28'),
              slice('2022-03-01','2022-03-31')]
axs = axs.ravel()

for i, ax in enumerate(axs):    
    colors = mdates.date2num(rh_mean.sel(time=time_slice[i]).time)
    tmp_temp = np.ma.masked_invalid(temp_diff.sel(time=time_slice[i]).values)
    tmp_vpd = np.ma.masked_invalid(vpd.sel(time=time_slice[i]).values)
    ax.scatter(tmp_temp, tmp_vpd, c=colors, s=1)
    ax.set_xlabel('T$_{inf}$ - T$_s$')
    ax.set_ylabel('Vapor Pressure Difference (Pa)')
    ax.set_title('VPD and Temperature Difference')
    norm = Normalize(vmin=colors.min(), vmax=colors.max())
    # choose a proper number of ticks to be shown on the colorbar
    N = 5
    ticks = np.linspace(colors.min(), colors.max(), N)
    # convert the tick back to date format
    labels = [mdates.num2date(t).strftime("%Y-%m-%d") for t in ticks]
    cb = fig.colorbar(
        matplotlib.cm.ScalarMappable(norm=norm, cmap='viridis'),
        ticks=ticks,
        ax=ax,
        label="Date"
    )
    cb.ax.set_yticklabels(labels);

In [ ]:
fig, axs = plt.subplots(4, figsize=(8,16), tight_layout=True)
time_slice = [slice('2021-12-01','2021-12-31'),
              slice('2022-01-01','2022-01-31'),
              slice('2022-02-01','2022-02-28'),
              slice('2022-03-01','2022-03-31')]
axs = axs.ravel()

for i, ax in enumerate(axs):    
    colors = mdates.date2num(rh_mean.sel(time=time_slice[i]).time)
    tmp_temp = np.ma.masked_invalid(temp_diff.sel(time=time_slice[i]).values)
    tmp_vpd = np.ma.masked_invalid(u_vpd.sel(time=time_slice[i]).values)
    ax.scatter(tmp_temp, tmp_vpd, c=colors, s=1)
    ax.set_xlabel('T$_{inf}$ - T$_s$')
    ax.set_ylabel('Vapor Pressure Difference * U (Pa * m/s)')
    ax.set_title('VPD*U and Temperature Difference')
    
    norm = Normalize(vmin=colors.min(), vmax=colors.max())
    # choose a proper number of ticks to be shown on the colorbar
    N = 5
    ticks = np.linspace(colors.min(), colors.max(), N)
    # convert the tick back to date format
    labels = [mdates.num2date(t).strftime("%Y-%m-%d") for t in ticks]
    cb = fig.colorbar(
        matplotlib.cm.ScalarMappable(norm=norm, cmap='viridis'),
        ticks=ticks,
        ax=ax,
        label="Date"
    )
    cb.ax.set_yticklabels(labels);

In [ ]:
t1 = '2021-12-23'
t2 = '2022-01-03'
time_slice = slice(t1, t2)

x = np.nan_to_num(temp_diff.sel(time=time_slice).values)
y = np.nan_to_num(rh_mean.sel(time=time_slice).values)

h =plt.hist2d(x, y, bins=25,cmap='plasma_r', vmax=500)
plt.colorbar(h[3])
plt.title('T$_{air}$ - T$_{s}$ vs RH'+ f'\n{t1} to {t2}')
plt.xlabel('T$_{air}$ - T$_{s}$ (C)')
plt.ylabel('RH')


In [ ]:
t1 = '2022-01-09'
t2 = '2022-01-20'
time_slice = slice(t1, t2)

x = np.nan_to_num(temp_diff.sel(time=time_slice).values)
y = np.nan_to_num(rh_mean.sel(time=time_slice).values)

h =plt.hist2d(x, y, bins=25,cmap='plasma_r', vmax=500)
plt.colorbar(h[3])
plt.title('T$_{air}$ - T$_{s}$ vs RH'+ f'\n{t1} to {t2}')
plt.xlabel('T$_{air}$ - T$_{s}$ (C)')
plt.ylabel('RH')

In [ ]:
t1 = '2022-01-22'
t2 = '2022-01-31'
time_slice = slice(t1, t2)

x = np.nan_to_num(temp_diff.sel(time=time_slice).values)
y = np.nan_to_num(rh_mean.sel(time=time_slice).values)

h =plt.hist2d(x, y, bins=25,cmap='plasma_r', vmax=500)
plt.colorbar(h[3])
plt.title('T$_{air}$ - T$_{s}$ vs RH'+ f'\n{t1} to {t2}')
plt.xlabel('T$_{air}$ - T$_{s}$ (C)')
plt.ylabel('RH')

In [ ]:
t1 = '2021-12-23'
t2 = '2021-12-31'
time_slice = slice(t1, t2)

x = np.nan_to_num(temp_diff.sel(time=time_slice).values)
y = np.nan_to_num(vpd.sel(time=time_slice).values)

h =plt.hist2d(x, y, bins=25,cmap='viridis_r', vmax=500)
plt.colorbar(h[3], label='Counts')
plt.title('T$_{air}$ - T$_{s}$ vs VPD'+ f'\n{t1} to {t2}', size=20)
plt.xlabel('T$_{air}$ - T$_{s}$ (C)', size=18)
plt.ylabel('Vapor Pressure Deficit (Pa)', size=18)

In [ ]:
t1 = '2022-01-20'
t2 = '2022-01-31'
time_slice = slice(t1, t2)

x = np.nan_to_num(temp_diff.sel(time=time_slice).values)
y = np.nan_to_num(vpd.sel(time=time_slice).values)

h =plt.hist2d(x, y, bins=25,cmap='viridis_r', vmax=500)
plt.colorbar(h[3], label='Counts')
plt.title('T$_{air}$ - T$_{s}$ vs VPD'+ f'\n{t1} to {t2}', size=20)
plt.xlabel('T$_{air}$ - T$_{s}$ (C)', size=18)
plt.ylabel('Vapor Pressure Deficit (Pa)', size=18)

In [ ]:
t1 = '2022-01-01'
t2 = '2022-02-01'
time_slice = slice(t1, t2)

x = np.nan_to_num(temp_diff.sel(time=time_slice).values)
y = np.nan_to_num(vpd.sel(time=time_slice).values)

h =plt.hist2d(x, y, bins=25,cmap='viridis_r', vmax=500)
plt.colorbar(h[3], label='Counts')
plt.title('T$_{air}$ - T$_{s}$ vs VPD'+ f'\n{t1} to {t2}', size=20)
plt.xlabel('T$_{air}$ - T$_{s}$ (C)', size=18)
plt.ylabel('Vapor Pressure Deficit (Pa)', size=18)

In [ ]:
t1 = '2022-02-01'
t2 = '2022-03-01'
time_slice = slice(t1, t2)

x = np.nan_to_num(temp_diff.sel(time=time_slice).values)
y = np.nan_to_num(vpd.sel(time=time_slice).values)

h =plt.hist2d(x, y, bins=25,cmap='viridis_r', vmax=500)
plt.colorbar(h[3], label='Counts')
plt.title('T$_{air}$ - T$_{s}$ vs VPD'+ f'\n{t1} to {t2}', size=20)
plt.xlabel('T$_{air}$ - T$_{s}$ (C)', size=18)
plt.ylabel('Vapor Pressure Deficit (Pa)', size=18)

In [ ]:
t1 = '2022-03-01'
t2 = '2022-04-01'
time_slice = slice(t1, t2)

x = np.nan_to_num(temp_diff.sel(time=time_slice).values)
y = np.nan_to_num(vpd.sel(time=time_slice).values)

h =plt.hist2d(x, y, bins=25,cmap='viridis_r', vmax=500)
plt.colorbar(h[3], label='Counts')
plt.title('T$_{air}$ - T$_{s}$ vs VPD'+ f'\n{t1} to {t2}', size=20)
plt.xlabel('T$_{air}$ - T$_{s}$ (C)', size=18)
plt.ylabel('Vapor Pressure Deficit (Pa)', size=18)

Mid-February was characterized by periods of light snow and windy conditions throughout the day. This period was cold as well. It appears there was an event during this time that was both windy, cold and dry. Potential for significant sublimation during this period.

In [ ]:
t1 = '2022-02-10'
t2 = '2022-02-24'
time_slice = slice(t1, t2)

x = np.nan_to_num(temp_diff.sel(time=time_slice).values)
y = np.nan_to_num(u_vpd.sel(time=time_slice).values)

h =plt.hist2d(x, y, bins=25,cmap='magma_r', vmax=500)
plt.colorbar(h[3])
plt.title('T$_{air}$ - T$_{s}$ vs VPD*U'+ f'\n{t1} to {t2}')
plt.xlabel('T$_{air}$ - T$_{s}$ (C)')
plt.ylabel('Vapor Pressure Deficit*U (Pa * m/s)')

In [ ]:
t1 = '2022-01-10'
t2 = '2022-01-25'
time_slice = slice(t1, t2)

x = np.nan_to_num(temp_diff.sel(time=time_slice).values)
y = np.nan_to_num(u_vpd.sel(time=time_slice).values)

h =plt.hist2d(x, y, bins=25,cmap='magma_r', vmax=500)
plt.colorbar(h[3])
plt.title('T$_{air}$ - T$_{s}$ vs VPD*U'+ f'\n{t1} to {t2}')
plt.xlabel('T$_{air}$ - T$_{s}$ (C)')
plt.ylabel('Vapor Pressure Deficit*U (Pa * m/s)')

This period was marked by periods of heavy snow, wind,m and cold temperatures. Possibily potential periods of substantial sublimation in between storm cells? Although the temperature deficit was not extreme, we do see 

In [ ]:
t1 = '2022-03-04'
t2 = '2022-03-14'
time_slice = slice(t1, t2)

x = np.nan_to_num(temp_diff.sel(time=time_slice).values)
y = np.nan_to_num(u_vpd.sel(time=time_slice).values)

h =plt.hist2d(x, y, bins=25,cmap='magma_r', vmax=500)
plt.colorbar(h[3])
plt.title('T$_{air}$ - T$_{s}$ vs VPD*U'+ f'\n{t1} to {t2}')
plt.xlabel('T$_{air}$ - T$_{s}$ (C)')
plt.ylabel('Vapor Pressure Deficit*U (Pa * m/s)')

In [ ]:
t1 = '2022-01-04'
t2 = '2022-01-16'
time_slice = slice(t1, t2)

x = np.nan_to_num(temp_diff.sel(time=time_slice).values)
y = np.nan_to_num(u_vpd.sel(time=time_slice).values)

h =plt.hist2d(x, y, bins=25,cmap='magma_r', vmax=500)
plt.colorbar(h[3])
plt.title('T$_{air}$ - T$_{s}$ vs VPD*U'+ f'\n{t1} to {t2}')
plt.xlabel('T$_{air}$ - T$_{s}$ (C)')
plt.ylabel('Vapor Pressure Deficit*U (Pa * m/s)')

In [ ]:
temps = np.linspace(-25,0,30)
es_ice = sat_vapor_pressure(temps)
es_water = sat_vapor_pressure_water(temps)
fig, ax = plt.subplots(figsize=(10,8))
ax.plot(temps,es_ice, label='Saturated Vapor Pressure Curve wrt Ice')
ax.plot(temps,es_water, label='Saturated Vapor Pressure Curve wrt Water')
h = ax.hist2d(met_ds['temp_mean'],e_air,bins=100, cmap='viridis_r');
# ax.scatter(t_surf,es_surf, s=0.2)
ax.set_xlim(-25,0)
fig.colorbar(h[3], ax=ax)
ax.legend()
ax.set_xlabel('Temperature (C)')
ax.set_ylabel('Vapor Pressure (Pa)')

In [ ]:
q_h = ecor_gts_ds['rho'] * ecor_gts_ds['cp'] * ecor_gts_ds['cvar_wt']
q_h = q_h.rename('qh')

In [ ]:
default_settings = {'height': 200, 'width': 500}

time_slice = slice('2022-03-08','2022-03-13')
sebs_gts_ds_subset = sw_rad.sel(time=time_slice)
surf_temp_da_subset = surf_t.sel(time=time_slice)
ecor_ds_subset = ecor_gts_ds.sel(time=time_slice)
q_h_subset = q_h.sel(time=time_slice)
met_ds_subset = met_ds.sel(time=time_slice)

sw_down = (sebs_gts_ds_subset['down_short_hemisp']/10).hvplot(label='Downwelling SW/10',**default_settings)
lv = q_h_subset.hvplot(label='Sensible Heat Flux (MOST)',**default_settings)

hs = ecor_ds_subset.where(ecor_ds_subset['qc_h']==0)['h'].hvplot(label='Sensible Heat Flux',
                                                                 ylabel='Sensible/Latent Heat Flux\n(W/m^2)',
                                                                 **default_settings)
temp = (met_ds_subset.where(met_ds_subset['qc_temp_mean']==0)['temp_mean']).hvplot(label='Temperature',
                                                                                ylabel='Temperature (C)',
                                                                                **default_settings,
                                                                                color='orange')          
surf_temp_plt = (surf_temp_da_subset-273.15).hvplot(label='Surface Temperature')                                                                                                                                                                                                                       
wind_spd = met_ds_subset.where(met_ds_subset['qc_wspd_arith_mean']==0)['wspd_arith_mean'].hvplot(label='Wind Speed',
                                                                                  ylabel='Wind Speed (m/s)',
                                                                                  **default_settings,
                                                                                  color='green')       
southern_winds=(ecor_ds_subset.where((ecor_ds_subset['wind_dir']>90) & (ecor_ds_subset['wind_dir']<270))['wind_dir']).hvplot.scatter(label='Wind Direction (SW-SE winds)',
                                                                                  ylabel='Degrees',
                                                                                  **default_settings,
                                                                                  color='black').opts(marker='x',legend_position='top', legend_offset=(0,5),legend_cols=1) 
northern_winds1=(ecor_ds_subset.where((ecor_ds_subset['wind_dir']>270))['wind_dir']-360).hvplot.scatter(
                                                                                  ylabel='Degrees',
                                                                                  **default_settings,
                                                                                  color='red').opts(marker='x',legend_position='top', legend_offset=(0,5),legend_cols=1)
northern_winds2=(ecor_ds_subset.where((ecor_ds_subset['wind_dir']<90))['wind_dir']).hvplot.scatter(label='Wind Direction (NW-NE winds)',
                                                                                  ylabel='Degrees',
                                                                                  **default_settings,
                                                                                  color='red').opts(marker='x',legend_position='top', legend_offset=(0,5),legend_cols=1)                                                                                                
((lv*hs*sw_down).opts(legend_position='top', legend_offset=(0,5),legend_cols=1)+(temp*surf_temp_plt).opts(legend_position='top', legend_offset=(0,5),legend_cols=1)+wind_spd+northern_winds1*southern_winds*northern_winds2).cols(2)                         

In [ ]:
t_dew = met_ds['temp_mean'] - ((100-met_ds['rh_mean'])/5)
t_dew = t_dew.rename('t_dew')

(t_dew.hvplot.hist(alpha=0.5, label='T_dew') * t_surf.hvplot.hist(alpha=0.5, label='T_surf'))